# Initial codes

This notebook illustrates how Bayesian inference can be used to infer response rates of each group (basket) in a Basket trial.

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
from os.path import exists

sys.path.append('..')
sys.path.append('.')

In [3]:
import numpy as np
import pandas as pd
import arviz as az
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [4]:
from pyBasket.common import load_obj
from pyBasket.common import (
    GROUP_STATUS_COMPLETED_EFFECTIVE,
    MODEL_INDEPENDENT_BERN,
    MODEL_BHM,
    MODEL_PYBASKET
)

In [5]:
def get_output_filenames(result_dir, scenario, with_clustering_info, n_clusters):
    base_filename = f'scenario_{scenario}_clustering_{with_clustering_info}'
    if with_clustering_info:
        base_filename += f'_ncluster_{n_clusters}'

    out_trial_results = os.path.join(result_dir, base_filename + '_trial_results.p')
    return out_trial_results

In [6]:
out_dir = os.path.abspath(os.path.join('..', 'scripts', 'results'))
out_dir

'/Users/joewandy/Work/git/pyBasket/scripts/results'

In [7]:
scenarios = range(0, 7)
clustering = [False, True]
n_clusters = [5, 10]

In [8]:
def gather_data(scenarios, clustering, n_clusters, out_dir):
    data = {}

    for scenario in scenarios:
        for cl in clustering:
            nc_values = [None] if not cl else n_clusters

            for nc in nc_values:
                try:
                    fname = get_output_filenames(out_dir, scenario, cl, nc)
                    print(fname)                
                    trial_results = load_obj(fname)
                    key = (scenario, cl, nc)
                    data[key] = trial_results
                except FileNotFoundError:
                    pass
                
    return data

In [9]:
data = gather_data(scenarios, clustering, n_clusters, out_dir)

/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_0_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_1_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_False_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_5_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_2_clustering_True_ncluster_10_trial_results.p
/Users/joewandy/Work/git/pyBasket/scripts/results/scenario_3_clustering_False_trial_results

In [10]:
def get_analysis(data, trial_idx, scenario, use_clustering, num_clusters, last_only=True):
    key = (scenario, use_clustering, num_clusters)
    analysis_results = {}

    if key not in data:
        print(f"No trial result found for key {key}")
        return

    trial_result = data[key]

    if trial_idx >= len(trial_result):
        print(f"Trial index {trial_idx} out of range.")
        return

    for analysis_name in trial_result[trial_idx].idfs:
        idfs = trial_result[trial_idx].idfs[analysis_name]
        if last_only and idfs:
            idfs = idfs[-1]
        analysis_results[analysis_name] = idfs

    return analysis_results

def get_analysis_by_name(analysis_results, analysis_name):
    if analysis_name not in analysis_results:
        print(f"No analysis results found for {analysis_name}")
        return

    idfs = analysis_results[analysis_name]

    return idfs


def display_analysis(idfs):
    if isinstance(idfs, list):
        for df in idfs:
            display(df)
    elif idfs is not None:
        display(idfs)
    else:
        print("No data to display.")


In [11]:
def get_true_arr(scenario):

    if scenario == 0:
        y_true = np.array([False, False, False, False, False, False])
    elif scenario == 1:
        y_true = np.array([True, False, False, False, False, False])
    elif scenario == 2:
        y_true = np.array([True, True, False, False, False, False])
    elif scenario == 3:
        y_true = np.array([True, True, True, False, False, False])
    elif scenario == 4:
        y_true = np.array([True, True, True, True, False, False])
    elif scenario == 5:
        y_true = np.array([True, True, True, True, True, False])
    elif scenario == 6:
        y_true = np.array([True, True, True, True, True, True])
        
    y_true_all = []
    for trial_idx in range(total):
        y_true_all.append(y_true)
    y_true_all = np.array(y_true_all)
    return y_true_all

In [12]:
def get_pred_arr(data, total, analysis_name, scenario, use_clustering, num_clusters):
    y_pred_all = []
    for trial_idx in range(total):

        # get the data first
        analysis_results = get_analysis(data, trial_idx, scenario, use_clustering, num_clusters)

        df = get_analysis_by_name(analysis_results, analysis_name)
        df_status = df['group_status'] == GROUP_STATUS_COMPLETED_EFFECTIVE
        y_pred = df_status.values
        y_pred_all.append(y_pred)

    y_pred_all = np.array(y_pred_all)
    return y_pred_all

|                   | Predicted Negative | Predicted Positive |
|-------------------|--------------------|--------------------|
| Actual Negative   |        TN          |        FP          |
| Actual Positive   |        FN          |        TP          |

In [13]:
def get_percent_reject(y_true_all, y_pred_all, basket_idx):

    y_true = y_true_all[:, basket_idx]
    y_pred = y_pred_all[:, basket_idx]
    
    # Calculate the basket-wise type I error rate. 
    # This corresponds to the proportion of false positives, or 
    # the cases when the null hypothesis is true (i.e., the treatment 
    # is not effective), but it's incorrectly rejected 
    # (leading us to wrongly believe that the treatment is effective).
    
    count = np.sum((y_true == False) & (y_pred == True))
    total = y_true_all.shape[0]
    final = f'I:{count / total * 100:.2f}'
    # final = count / total * 100
        
    if count == 0: # 
        
        # If we can't calculate the type I error, then report the power instead
        # The basket-wise power corresponds to the proportion of true positives, 
        # or the cases when the null hypothesis is false (i.e., the treatment is 
        # effective), and it's correctly rejected (leading us to correctly conclude 
        # that the treatment is effective).
        
        count = np.sum((y_true == True) & (y_pred == True))    
        total = y_true_all.shape[0]
        final = f'p:{count / total * 100:.2f}'
        # final = count / total * 100
        
    return final

In [14]:
def get_metrics(y_true_all, y_pred_all, basket_idx):
    cm = confusion_matrix(y_true_all[:, basket_idx], y_pred_all[:, basket_idx], labels=[False, True])
    TN = float(cm[0][0])
    FP = float(cm[0][1])
    FN = float(cm[1][0])
    TP = float(cm[1][1])
    accuracy = (TN + TP) / (TN + FP + FN + TP)

    try:
        precision = TP / (TP + FP)
    except ZeroDivisionError:
        precision = 0.0

    try:
        recall = TP / (TP + FN)
    except ZeroDivisionError:
        recall = 0.0

    try:
        sensitivity = TN / (TN + FP)
    except ZeroDivisionError:
        sensitivity = 0.0

    try:
        f1 = 2 * (precision * recall) / (precision + recall)
    except ZeroDivisionError:
        f1 = 0.0
        
    try:
        fpr = FP / (FP + TN)
    except ZeroDivisionError:
        fpr = 0.0
    
    return TN, FP, FN, TP, accuracy, precision, recall, sensitivity, f1, fpr

In [15]:
def analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters):
    df_data = []
    for analysis_name in analysis_names:
        for scenario_idx in range(n_scenario):
            y_true_all = get_true_arr(scenario_idx)
            y_pred_all = get_pred_arr(data, total, analysis_name, scenario_idx, use_clustering, num_clusters)

            for basket_idx in range(n_baskets):
                reject = get_percent_reject(y_true_all, y_pred_all, basket_idx)
                # TN, FP, FN, TP, accuracy, precision, recall, sensitivity, f1, fpr = get_metrics(y_true_all, y_pred_all, basket_idx)
                row = [scenario_idx, analysis_name, basket_idx, reject]
                df_data.append(row)

    reject_df = pd.DataFrame(df_data, columns=['scenario', 'analysis_name', 'basket_idx', 'reject'])
    reshaped_df = reject_df.pivot(index=['scenario', 'analysis_name'], columns='basket_idx', values='reject')
    return reshaped_df


In [16]:
total = 500
n_scenario = 7
n_baskets = 6
analysis_names = [
    MODEL_INDEPENDENT_BERN,
    MODEL_BHM,
    MODEL_PYBASKET
]

In [17]:
use_clustering = False
num_clusters = None
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0        1        2        3        4  \
scenario analysis_name                                                   
0        BHM               I:10.00  I:10.40  I:10.20  I:10.20  I:10.20   
         independent_bern   I:8.40  I:10.80  I:11.40  I:10.80   I:9.80   
         pyBasket           I:9.80  I:10.60  I:10.80  I:10.20   I:9.40   
1        BHM               p:55.60  I:30.40  I:30.60  I:31.00  I:31.40   
         independent_bern  p:84.20   I:9.60  I:12.20   I:9.60   I:9.40   
         pyBasket          p:76.00   I:5.20   I:8.40   I:4.80   I:5.00   
2        BHM               p:84.80  p:86.60  I:48.20  I:49.20  I:50.20   
         independent_bern  p:83.80  p:85.40   I:8.00   I:9.80  I:11.80   
         pyBasket          p:64.80  p:67.80   I:1.20   I:2.20   I:2.00   
3        BHM               p:90.80  p:92.20  p:92.20  I:58.40  I:56.20   
         independent_bern  p:80.80  p:80.60  p:82.40   I:9.20  I:10.20   
         pyBasket          p:52.80  p:53.00  p:52.60   p:0.00   I:1.40   
4        BHM               p:96.80  p:96.60  p:96.20  p:96.80  I:69.60   
         independent_bern  p:81.80  p:82.40  p:81.80  p:86.00  I:10.00   
         pyBasket          p:37.60  p:39.80  p:37.60  p:41.20   I:0.20   
5        BHM               p:98.40  p:99.00  p:99.40  p:98.80  p:98.20   
         independent_bern  p:81.00  p:82.00  p:85.20  p:79.60  p:80.00   
         pyBasket          p:27.00  p:27.20  p:30.00  p:27.60  p:26.40   
6        BHM               p:99.80  p:99.40  p:99.80  p:99.80  p:99.40   
         independent_bern  p:84.00  p:82.60  p:82.00  p:82.20  p:79.20   
         pyBasket          p:18.20  p:17.60  p:16.20  p:20.00  p:14.80   

basket_idx                        5  
scenario analysis_name               
0        BHM                I:10.00  
         independent_bern    I:9.00  
         pyBasket            I:9.60  
1        BHM                I:31.20  
         independent_bern    I:8.80  
         pyBasket            I:5.20  
2        BHM                I:51.80  
         independent_bern    I:9.20  
         pyBasket            I:1.40  
3        BHM                I:56.40  
         independent_bern    I:8.60  
         pyBasket            I:0.40  
4        BHM                I:71.40  
         independent_bern    I:9.20  
         pyBasket            p:0.00  
5        BHM                I:85.80  
         independent_bern    I:9.60  
         pyBasket            p:0.00  
6        BHM               p:100.00  
         independent_bern   p:83.00  
         pyBasket           p:17.20

In [18]:
use_clustering = True
num_clusters = 5
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0        1        2        3        4  \
scenario analysis_name                                                   
0        BHM                p:0.00   I:0.20   p:0.00   p:0.00   p:0.00   
         independent_bern   p:0.00   I:0.20   p:0.00   p:0.00   p:0.00   
         pyBasket           p:0.00   I:1.40   I:0.20   I:0.60   I:0.60   
1        BHM                p:1.60   p:0.00   p:0.00   I:0.20   p:0.00   
         independent_bern   p:7.80   I:0.20   I:0.40   I:0.20   I:0.40   
         pyBasket          p:12.40   I:0.40   I:0.60   I:0.80   I:1.80   
2        BHM                p:3.00  p:13.40   I:1.00   I:1.20   I:0.80   
         independent_bern  p:10.60  p:50.60   I:3.20   I:2.80   I:1.00   
         pyBasket          p:12.20  p:58.00   I:4.80   I:5.80   I:1.60   
3        BHM                p:0.60   p:1.60   p:1.20   p:0.00   p:0.00   
         independent_bern   p:5.40  p:12.20   p:8.80   p:0.00   I:0.40   
         pyBasket           p:6.20  p:21.40  p:15.60   I:0.20   I:1.00   
4        BHM                p:0.20   p:0.40   p:0.40   p:0.20   p:0.00   
         independent_bern   p:2.20   p:2.40   p:3.20   p:0.80   p:0.00   
         pyBasket           p:4.40   p:8.00   p:6.20   p:2.40   I:1.00   
5        BHM               p:79.20  p:78.40  p:78.00  p:80.40  p:80.40   
         independent_bern  p:67.80  p:62.40  p:59.20  p:67.40  p:70.40   
         pyBasket          p:46.20  p:38.40  p:42.40  p:42.80  p:53.20   
6        BHM                p:8.60   p:8.60   p:7.40   p:7.60   p:7.80   
         independent_bern  p:20.60  p:22.00  p:16.00  p:14.00  p:15.40   
         pyBasket          p:14.20  p:16.40   p:9.80   p:6.80   p:7.80   

basket_idx                       5  
scenario analysis_name              
0        BHM                p:0.00  
         independent_bern   p:0.00  
         pyBasket           p:0.00  
1        BHM                p:0.00  
         independent_bern   I:0.20  
         pyBasket           I:1.60  
2        BHM                I:1.00  
         independent_bern   I:2.20  
         pyBasket           I:6.00  
3        BHM                p:0.00  
         independent_bern   I:0.20  
         pyBasket           I:0.80  
4        BHM                I:0.20  
         independent_bern   I:0.40  
         pyBasket           I:1.00  
5        BHM               I:72.20  
         independent_bern  I:10.60  
         pyBasket           I:3.80  
6        BHM                p:6.80  
         independent_bern  p:10.80  
         pyBasket           p:8.60

In [19]:
use_clustering = True
num_clusters = 10
df = analyze_rejections(data, total, n_scenario, n_baskets, analysis_names, use_clustering, num_clusters)
df

basket_idx                       0        1        2        3        4  \
scenario analysis_name                                                   
0        BHM                I:1.20   I:1.20   I:0.40   I:0.20   I:0.20   
         independent_bern   I:6.00   I:8.20   I:5.40   I:8.00   I:5.00   
         pyBasket           I:7.20   I:7.60   I:6.80   I:5.80   I:5.60   
1        BHM                p:6.80   I:0.40   I:1.20   I:0.60   I:1.00   
         independent_bern  p:36.40   I:3.00   I:6.20   I:4.00   I:6.20   
         pyBasket          p:35.40   I:2.60   I:5.40   I:4.00   I:5.20   
2        BHM               p:17.20  p:18.00   I:3.20   I:4.80   I:4.00   
         independent_bern  p:45.20  p:51.20   I:3.40   I:7.40   I:4.60   
         pyBasket          p:41.40  p:47.60   I:3.00   I:7.80   I:2.40   
3        BHM               p:24.40  p:20.80  p:17.00   I:6.80   I:7.00   
         independent_bern  p:59.20  p:41.00  p:35.20   I:7.60   I:5.80   
         pyBasket          p:53.40  p:33.00  p:28.20   I:5.40   I:4.60   
4        BHM               p:32.60  p:39.60  p:33.60  p:37.00  I:22.40   
         independent_bern  p:44.20  p:63.80  p:49.20  p:57.20   I:6.40   
         pyBasket          p:34.40  p:53.80  p:39.60  p:51.20   I:3.40   
5        BHM               p:16.00  p:16.20  p:14.40  p:13.60  p:12.40   
         independent_bern  p:32.80  p:34.00  p:32.20  p:32.40  p:27.80   
         pyBasket          p:28.00  p:24.80  p:25.80  p:25.20  p:21.60   
6        BHM               p:31.40  p:36.00  p:26.60  p:30.60  p:28.40   
         independent_bern  p:43.60  p:56.60  p:23.60  p:37.60  p:26.40   
         pyBasket          p:35.60  p:50.80  p:16.80  p:33.60  p:22.60   

basket_idx                       5  
scenario analysis_name              
0        BHM                I:0.60  
         independent_bern   I:7.60  
         pyBasket           I:6.80  
1        BHM                I:0.20  
         independent_bern   I:3.20  
         pyBasket           I:3.80  
2        BHM                I:3.20  
         independent_bern   I:4.20  
         pyBasket           I:2.60  
3        BHM                I:7.00  
         independent_bern   I:4.60  
         pyBasket           I:3.60  
4        BHM               I:22.60  
         independent_bern   I:7.00  
         pyBasket           I:5.00  
5        BHM                I:7.20  
         independent_bern   I:3.40  
         pyBasket           I:2.60  
6        BHM               p:28.80  
         independent_bern  p:27.80  
         pyBasket          p:24.60